调包

In [ ]:
import cv2 as cv
import argparse
from random import randint
import time
import os

In [ ]:
parser = argparse.ArgumentParser(description='')
parser.add_argument('--input', type=str, help='Path to a video or a sequence of image.', default = 'E:/科研/视频和分析_11.6/20221103/00116.MTS')
args = parser.parse_args()

初始化

In [ ]:
vecPoints = []
bboxes = []
colors = []

video = cv.VideoCapture(cv.samples.findFileOrKeep(args.input))

完成输出设置

In [ ]:
time_t = time.strftime('%Y.%m.%d %H-%M-%S', time.localtime(time.time()))
outDir = 'E:/科研/视频和分析_' + time_t
os.mkdir(outDir)
outFile_1 = outDir + '\\videoTracker.avi'
s = (int(video.get(cv.CAP_PROP_FRAME_WIDTH)), int(video.get(cv.CAP_PROP_FRAME_HEIGHT)))
r = video.get(cv.CAP_PROP_FPS)
write = cv.VideoWriter(outFile_1, 0, r, (520,500), True)

In [ ]:
print('>> 请按空格开始截取图片')

while True:
    success, frame = video.read()
    frame = frame[520:1060,160:700]
    cv.imshow('Tracker', frame)
    write.write(frame)
    if cv.waitKey(1) == ord(' '):
        break

In [ ]:
while True:
    # draw bounding boxes over objects
    # selectROI's default behaviour is to draw box starting from the center
    # when fromCenter is set to false, you can draw box starting from top left corner
    bbox = cv.selectROI('Tracker', frame)
    bboxes.append(bbox)
    colors.append((randint(64, 255), randint(64, 255), randint(64, 255)))
    print("Press q to quit selecting boxes and start tracking")
    print("Press any other key to select next object")
    k = cv.waitKey(0) & 0xFF
    if (k == 113):  # q is pressed
        break

In [ ]:
# Create MultiTracker object
multiTracker = cv.legacy.MultiTracker.create()

In [ ]:
# Initialize MultiTracker
for bbox in bboxes:
    multiTracker.add(cv.legacy.TrackerBoosting_create(), frame, bbox)
    temp = []
    vecPoints.append(temp)

    # Process video and track objects
    while video.isOpened():
        success, frame = video.read()
        frame = frame[520:1060,160:700]
        if not success:
            break
 
        print('before-' + str(id(frame)))
 
        # get updated location of objects in subsequent frames
        success, boxes = multiTracker.update(frame)
 
        # draw tracked objects
        for i, newbox in enumerate(boxes):
            p1 = (int(newbox[0]), int(newbox[1]))
            p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
            # 画框，cv.rectangle(img, 坐标1, 坐标2, color, 边框宽度)
            cv.rectangle(frame, p1, p2, colors[i], 2, 1)

        for i, newbox in enumerate(boxes):
            # 标出画线位置的点
            vecPoints[i].append((int(newbox[0] + (newbox[2] * 0.5)) * 2, int(newbox[1] + (newbox[3] * 0.5)) * 2))
        if len(vecPoints) > 0:
            for i in range(len(vecPoints)):
                for j in range(len(vecPoints[i]) - 1):
                    # 画线，cv.line(img, pt1, pt2, color, thickness, lineType, shift)
                    # pt1是开始的点，pt2是结束的点
                    cv.line(frame, vecPoints[i][j], vecPoints[i][j + 1], colors[i], 2, 8, 1)
                    
        for i, newbox in enumerate(boxes):
            cv.putText(frame, 'id_' + str(i + 1), (int(newbox[0]), int(newbox[1]) - 3), cv.FONT_HERSHEY_PLAIN,1, colors[i], 1)
             
        # show frame    
        write.write(frame)                   
        cv.imshow('Tracker', frame)            
        
        print('after-' + str(id(frame)))
        
        # quit on ESC button
        if cv.waitKey(1) == ord('q'):
            break